In [1]:
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import os
import glob

In [2]:
def load_model():
    image_processor = AutoImageProcessor.from_pretrained('ashish-001/deepfake-detection-using-ViT')
    model = AutoModelForImageClassification.from_pretrained('ashish-001/deepfake-detection-using-ViT')
    return image_processor, model

In [ ]:
def predict_batch_images(image_folder, image_processor, model, extensions=['jpg', 'jpeg', 'png', 'bmp']):
    """Predict multiple images from a folder"""
    results = []
    
    # Get all image files
    image_files = []
    for ext in extensions:
        image_files.extend(glob.glob(os.path.join(image_folder, f"*.{ext}")))
        image_files.extend(glob.glob(os.path.join(image_folder, f"*.{ext.upper()}")))
    
    print(f"Found {len(image_files)} images to process")
    
    for i, image_path in enumerate(image_files, 1):
        print(f"Processing image {i}/{len(image_files)}: {os.path.basename(image_path)}")
        
        label, confidence = predict_single_image(image_path, image_processor, model)
        
        if label is not None:
            results.append({
                'image_path': image_path,
                'filename': os.path.basename(image_path),
                'prediction': label,
                'confidence': confidence
            })
            print(f"  Result: {label} (confidence: {confidence:.3f})")
        else:
            print(f"  Failed to process")
    
    return results

In [ ]:
def save_results(results, output_file='deepfake_results.txt'):
    """Save results to a text file"""
    with open(output_file, 'w') as f:
        f.write("Deepfake Detection Results\n")
        f.write("=" * 50 + "\n\n")
        
        for result in results:
            f.write(f"File: {result['filename']}\n")
            f.write(f"Prediction: {result['prediction']}\n")
            f.write(f"Confidence: {result['confidence']:.3f}\n")
            f.write("-" * 30 + "\n")
    
    print(f"Results saved to {output_file}")

In [ ]:
image_processor, model = load_model()

In [ ]:
image_folder = "./images"